---
title: 'STATS 202 Homework 3'
project:
  output-dir: output
format: pdf
author: "Alistair Keiller"
execute: 
  enabled: true
---

In [1]:
import numpy as np
import pandas as pd
import os
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import csv

sns.set_theme()

In [2]:
path = "FinalProjectData"

data = pd.concat(
    [pd.read_csv(os.path.join(path, f)) for f in os.listdir(path) if f.endswith(".csv")]
)
data["PSUM"] = np.sum([data["P" + str(n)] for n in range(1, 8)], axis=0)
data["NSUM"] = np.sum([data["N" + str(n)] for n in range(1, 8)], axis=0)
data["GSUM"] = np.sum([data["G" + str(n)] for n in range(1, 17)], axis=0)
data["Treatment"] = data["TxGroup"] == "Treatment"
data["Flagged"] = data["LeadStatus"] != "Passed"
passed_data = data[data["LeadStatus"] == "Passed"]
data

,Study,Country,PatientID,SiteID,RaterID,AssessmentID,TxGroup,VisitDay,P1,P2,...,G14,G15,G16,PANSS_Total,LeadStatus,PSUM,NSUM,GSUM,Treatment,Flagged
0,A,USA,10001,20035,30076,100679,Control,0,5,5,...,3,3,5,107,Assign to CS,30,24,53,False,True
1,A,USA,10001,20035,30076,101017,Control,11,5,5,...,3,3,5,109,Assign to CS,30,25,54,False,True
2,A,USA,10001,20035,30076,102177,Control,18,4,4,...,2,3,4,91,Passed,23,24,44,False,False
3,A,USA,10001,20035,30076,101533,Control,25,3,3,...,2,3,4,80,Flagged,19,22,39,False,True
4,A,USA,10001,20035,30076,100930,Control,39,3,3,...,2,3,4,77,Flagged,19,21,37,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2943,D,China,40223,80001,120013,400496,Treatment,17,4,3,...,2,4,4,79,Flagged,18,21,40,True,True
2944,D,China,40223,80001,120013,402261,Treatment,21,4,3,...,2,3,3,71,Flagged,18,18,35,True,True
2945,D,China,40223,80001,120013,400605,Treatment,28,4,3,...,2,3,3,74,Flagged,19,19,36,True,True
2946,D,China,40223,80001,120013,402460,Treatment,42,4,3,...,2,4,3,77,Flagged,19,20,38,True,True


# 1 Treatment effect

We will clearly define our tests before observing or analyzing the data to simulate the regulations that prevent data snooping in regulated industries. The goal is to determine if the treatment ( $\text{TxGroup} == \text{Treatment}$, which will be our preprocessing step ) has an effect on schizophrenia over time, as measured by the PANSS score ( $\text{PANSS\_Total}$ ). So, we want to see if there is a change in $\text{PANSS\_Total}$ over time, which can be determined by looking for a correlation between $\text{VisitDay}$ and $\text{PANSS\_Total}$ using a linear regression model:

$$\text{PANSS\_Total}=\hat{\beta_0}+\hat{\beta_1}\text{VisitDay}$$

Our null hypothesis is that the treatment has no effect, and we can reject the null hypothesis if and only if we find that $\hat{\beta_1}\neq 0$ with a $p<0.05$.

In [3]:
smf.ols("PANSS_Total ~ VisitDay*Treatment", data=passed_data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            PANSS_Total   R-squared:                       0.396
Model:                            OLS   Adj. R-squared:                  0.396
Method:                 Least Squares   F-statistic:                     3467.
Date:                Sat, 29 Jul 2023   Prob (F-statistic):               0.00
Time:                        02:05:26   Log-Likelihood:                -64467.
No. Observations:               15841   AIC:                         1.289e+05
Df Residuals:                   15837   BIC:                         1.290e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     82.9635      0.222    374.211      0.000      82.529      83.398
Treatment[T.True]             -1.0590      0.320     -3.310      0.001      -1.686      -0.432
VisitDay                      -0.1177      0.002    -73.833      0.000      -0.121      -0.115
VisitDay:Treatment[T.True]     0.0053      0.002      2.330      0.020       0.001       0.010
==============================================================================
Omnibus:                      418.754   Durbin-Watson:                   0.530
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              497.541
Skew:                           0.351   Prob(JB):                    9.13e-109
Kurtosis:                       3.512   Cond. No.                         522.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [4]:
sns.lmplot(
    x="VisitDay",
    y="PANSS_Total",
    data=passed_data,
    hue="TxGroup",
)
plt.show()

/opt/homebrew/lib/python3.11/site-packages/seaborn/axisgrid.py:118: UserWarning:

The figure layout has changed to tight



<Figure size 1872x1500 with 1 Axes>

In [5]:
smf.ols("PSUM ~ VisitDay*Treatment", data=passed_data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   PSUM   R-squared:                       0.332
Model:                            OLS   Adj. R-squared:                  0.332
Method:                 Least Squares   F-statistic:                     2627.
Date:                Sat, 29 Jul 2023   Prob (F-statistic):               0.00
Time:                        02:05:27   Log-Likelihood:                -48160.
No. Observations:               15841   AIC:                         9.633e+04
Df Residuals:                   15837   BIC:                         9.636e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     19.9753      0.079    252.221      0.000      19.820      20.131
Treatment[T.True]             -0.3030      0.114     -2.652      0.008      -0.527      -0.079
VisitDay                      -0.0366      0.001    -64.317      0.000      -0.038      -0.036
VisitDay:Treatment[T.True]     0.0017      0.001      2.087      0.037       0.000       0.003
==============================================================================
Omnibus:                      786.417   Durbin-Watson:                   0.509
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              907.748
Skew:                           0.571   Prob(JB):                    7.67e-198
Kurtosis:                       3.269   Cond. No.                         522.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
sns.lmplot(
    x="VisitDay",
    y="PSUM",
    data=passed_data,
    hue="TxGroup",
)
plt.show()

/opt/homebrew/lib/python3.11/site-packages/seaborn/axisgrid.py:118: UserWarning:

The figure layout has changed to tight



<Figure size 1872x1500 with 1 Axes>

In [7]:
smf.ols("NSUM ~ VisitDay*Treatment", data=passed_data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   NSUM   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     1291.
Date:                Sat, 29 Jul 2023   Prob (F-statistic):               0.00
Time:                        02:05:28   Log-Likelihood:                -47217.
No. Observations:               15841   AIC:                         9.444e+04
Df Residuals:                   15837   BIC:                         9.447e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     22.2323      0.075    297.944      0.000      22.086      22.379
Treatment[T.True]             -0.1657      0.108     -1.539      0.124      -0.377       0.045
VisitDay                      -0.0238      0.001    -44.299      0.000      -0.025      -0.023
VisitDay:Treatment[T.True]     0.0003      0.001      0.367      0.713      -0.001       0.002
==============================================================================
Omnibus:                      166.462   Durbin-Watson:                   0.511
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              181.395
Skew:                           0.219   Prob(JB):                     4.08e-40
Kurtosis:                       3.287   Cond. No.                         522.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
sns.lmplot(
    x="VisitDay",
    y="NSUM",
    data=passed_data,
    hue="TxGroup",
)
plt.show()

/opt/homebrew/lib/python3.11/site-packages/seaborn/axisgrid.py:118: UserWarning:

The figure layout has changed to tight



<Figure size 1872x1500 with 1 Axes>

In [9]:
smf.ols("GSUM ~ VisitDay*Treatment", data=passed_data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   GSUM   R-squared:                       0.361
Model:                            OLS   Adj. R-squared:                  0.361
Method:                 Least Squares   F-statistic:                     2979.
Date:                Sat, 29 Jul 2023   Prob (F-statistic):               0.00
Time:                        02:05:29   Log-Likelihood:                -54166.
No. Observations:               15841   AIC:                         1.083e+05
Df Residuals:                   15837   BIC:                         1.084e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     40.7559      0.116    352.228      0.000      40.529      40.983
Treatment[T.True]             -0.5903      0.167     -3.535      0.000      -0.918      -0.263
VisitDay                      -0.0573      0.001    -68.877      0.000      -0.059      -0.056
VisitDay:Treatment[T.True]     0.0033      0.001      2.799      0.005       0.001       0.006
==============================================================================
Omnibus:                      500.923   Durbin-Watson:                   0.584
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              580.379
Skew:                           0.406   Prob(JB):                    9.38e-127
Kurtosis:                       3.470   Cond. No.                         522.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
sns.lmplot(
    x="VisitDay",
    y="GSUM",
    data=passed_data,
    hue="TxGroup",
)
plt.show()

/opt/homebrew/lib/python3.11/site-packages/seaborn/axisgrid.py:118: UserWarning:

The figure layout has changed to tight



<Figure size 1872x1500 with 1 Axes>

We can say with statistical significance that the control treatment is more effective than the new treatment, with a $\hat{\beta'}_1\approx -0.1208$ versus $\hat{\beta}_1\approx -0.1134$, however, that does not mean that the new treatment is ineffective. For example, if it has other advantages like few side effects or lower cost, preventing the control treatment from being an option, this study showing the efficacy of the new treatment could be critical information. Overall, it is not more effective / an improvement over the control, but based on existing research it is significantly better than untreated Schizophrenia, which often gets worse over time. However, **it is effective**.

**ADD MORE ANALISYS HERE** talk about how the inital treatment group tends to be less schizophrenic than the control group. Solving this could be the object of a future study that is randmly assigns control and treatment such that the intial values are identical, becuase trying to artifically normilize them in this study, given that the initial groups are statistically significantly different, will result in other ( possibly unseen ) artifacts.

# 2 Patient segmentation

First, we need to preprocess the data. We want to filter the data only to consider initial measurements of patients, and we want to scale each of the variables used in the evaluation to zero means and unit variance for equal emphasis on more and less sensitive measurements.

In [11]:
first_day_passed_data = passed_data[passed_data["VisitDay"] == 0]
numerical_first_day_passed_data = first_day_passed_data.drop(
    [
        "Study",
        "Country",
        "PatientID",
        "SiteID",
        "RaterID",
        "AssessmentID",
        "TxGroup",
        "VisitDay",
        "PANSS_Total",
        "LeadStatus",
        "Flagged"
    ],
    axis=1,
)
scaled_numerical_first_day_passed_data = StandardScaler().fit_transform(
    numerical_first_day_passed_data
)

Also, let's check if there are many double first day measurments to see if that could skew the data

In [12]:
len(first_day_passed_data["PatientID"])

1747

In [13]:
len(np.unique(first_day_passed_data["PatientID"]))

1745

Nope, that is not a problem so we can ignore that as a possible skew of the data.

Now we can run a highirichical clustering to create a dendrogram since our dataset is relatively small

In [14]:
dendrogram(
    linkage(scaled_numerical_first_day_passed_data, method="average"),
    truncate_mode="level",
    p=10,
    no_labels=True,
    color_threshold=10.8,
)
plt.show()

<Figure size 1650x1050 with 1 Axes>

$3$ seems like the most sensible value. To confirm these values, let's run a KMeans $n$ vs. $\text{error}$ to confirm this cluster size.

In [15]:
sns.lineplot(
    x=range(1, 10),
    y=[
        KMeans(n_clusters=n, random_state=42, n_init="auto")
        .fit(scaled_numerical_first_day_passed_data)
        .inertia_
        for n in range(1, 10)
    ],
)
plt.show()

<Figure size 1650x1050 with 1 Axes>

The new graph confirms that $3$ is a valid choice

In [16]:
n_clusters = 3
first_day_passed_data["Cluster"] = (
    KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
    .fit(scaled_numerical_first_day_passed_data)
    .labels_
)

/var/folders/8f/dmypnd_j6h5gwtjwt90wyw680000gn/T/ipykernel_90181/3284583442.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [17]:
first_day_passed_data.groupby("Cluster")[
    ["PANSS_Total", "PSUM", "NSUM", "GSUM"]
].mean().reset_index()

,Cluster,PANSS_Total,PSUM,NSUM,GSUM
0,0,80.650155,17.224458,23.823529,39.602167
1,1,102.295938,25.427466,26.833656,50.034816
2,2,83.150685,23.109589,18.977740,41.063356


In [18]:
country_cluster_counts = first_day_passed_data[["Country", "Cluster"]].value_counts()
cluster_counts = first_day_passed_data["Cluster"].value_counts()
plt.figure(figsize=(10, 6))
sns.barplot(
    x="Country",
    y="Percentage",
    hue="Cluster",
    data=(
        country_cluster_counts
        / [
            cluster_counts[i] / 100
            for i in country_cluster_counts.reset_index()["Cluster"]
        ]
    ).reset_index(name="Percentage"),
)
plt.xticks(rotation=90)
plt.show()

<Figure size 3000x1800 with 1 Axes>

In [19]:
PANSS_Total_cluster_counts = first_day_passed_data[
    ["PANSS_Total", "Cluster"]
].value_counts()
cluster_counts = first_day_passed_data["Cluster"].value_counts()
plt.figure(figsize=(10, 6))
sns.barplot(
    x="PANSS_Total",
    y="Percentage",
    hue="Cluster",
    data=(
        PANSS_Total_cluster_counts
        / [
            cluster_counts[i] / 100
            for i in PANSS_Total_cluster_counts.reset_index()["Cluster"]
        ]
    ).reset_index(name="Percentage"),
)
plt.xticks(rotation=90)
plt.show()

<Figure size 3000x1800 with 1 Axes>

The clusters that form are
- **Cluster 1:** Exhibit slightly more positive and significantly more negative and general schizophrenic metrics than average
- **Cluster 2:** Exhibit significantly less positive, negative, and general schizophrenic metrics than average
- **Cluster 3:** Exhibit significantly more positive, and slightly more negative and general schizophrenic metrics than average
- **Cluster 4:** Exhibit less positive, a tiny bit less general, and more negative schizophrenic metrics than average
- **Cluster 5:** Exhibit slightly more positive, less negative, and the tiniest bit less general metrics than average.
In these groups, groups $1$ and $3$ would be the most important to look into because they will be the most likely to have schizophrenia-related problems, and this shows approximately what type of problems may arise in those groups ( the generally more negative problem groups with still a significant amount of positive and general, and more positive groups with a significant amount of general and less negative ). This also provides the basis for further investigation into the specifics of each metric. However, as a simple clustering algorithm, these results should be taken lightly not to mean that groups $2$ and $4$ are the only important ones.

# 3 Forecasting



In [20]:
data_e = data[data["Study"] == "E"]
data_e = data_e.sort_values("VisitDay")
data_e_patients = np.unique(data_e["PatientID"])
submission = pd.DataFrame(
    {
        "PatientID": data_e_patients,
        "PANSS_Total": [
            data_e[data_e["PatientID"] == PatientID]["PANSS_Total"].iloc[-1]
            for PatientID in data_e_patients
        ],
    }
)
submission.to_csv("week18.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

In [21]:
submission = pd.DataFrame(
    {
        "PatientID": data_e_patients,
        "PANSS_Total": [
            make_pipeline(StandardScaler(), SVR())
            .fit(
                data_e[data_e["PatientID"] == PatientID][["VisitDay"]],
                data_e[data_e["PatientID"] == PatientID]["PANSS_Total"],
            )
            .predict(
                pd.DataFrame(
                    {
                        "VisitDay": [
                            data_e[data_e["PatientID"] == PatientID]["VisitDay"].iloc[
                                -1
                            ]
                            + 7
                        ]
                    }
                )
            )[0]
            for PatientID in data_e_patients
        ],
    }
)
submission.to_csv("week18.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

# 4 Classification

In [22]:
lr = make_pipeline(StandardScaler(), LogisticRegression()).fit(data.drop(["Study","Country","TxGroup","LeadStatus","Flagged"], axis=1), data["Flagged"])
submission = pd.DataFrame(
    {
        "AssessmentID": data_e["AssessmentID"],
        "LeadStatus": [x for x,y in lr.predict_proba(data_e.drop(["Study","Country","TxGroup","LeadStatus","Flagged"], axis=1))]
    }
)
submission.to_csv("class.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

